### pymupdf4llm + OCR (py-zerox) 사용

정성적인 결과가 가장 좋음.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import asyncio
import textwrap
from copy import deepcopy
from pathlib import Path

import pymupdf4llm
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters.markdown import MarkdownHeaderTextSplitter
from pyzerox import zerox
from pyzerox.core.types import ZeroxOutput

/Users/kyuyeon/opt/miniconda3/envs/msbs/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [3]:
DATASET_DIR = "dataset"
MODEL_NAME = "gpt-4o"
OCR_CUSTOM_SYS_PROMPT = """
    The provided image document is a manual for specific equipment, primarily written in Korean.
    Convert the content of the PDF page into Markdown format.
    Ensure all text, tables, and formatting are fully and accurately represented.
    Format tables clearly and correctly for Markdown, ensuring proper alignment.
    Do not use code blocks in the Markdown output.
    Provide only the converted Markdown content without any explanations or additional comments.
"""
EMBEDDING_MODEL = "text-embedding-3-large"

In [4]:
def strip_prompt(prompt: str) -> str:
    return textwrap.dedent(prompt).strip()


async def zerox_parse(file_name: str) -> ZeroxOutput:
    print(f"Processing {file_name} with zerox...")
    zerox_output = await zerox(
        file_path=file_name,
        model=MODEL_NAME,
        custom_system_prompt=strip_prompt(OCR_CUSTOM_SYS_PROMPT),
    )
    zerox_output.file_name = file_name
    return zerox_output

In [5]:
pdf_paths = list(Path(DATASET_DIR).rglob("DATA*.pdf"))
pdf_paths = sorted([str(pdf_path) for pdf_path in pdf_paths])

In [6]:
zerox_results = [zerox_parse(pdf_path) for pdf_path in pdf_paths]
zerox_results_gathered = await asyncio.gather(*zerox_results)

Processing dataset/DATA1. KT420(L) - 조작설명서 (MITSUBISH)_17 0420 - 완료.pdf with zerox...


/Users/kyuyeon/opt/miniconda3/envs/msbs/lib/python3.11/site-packages/pyzerox/models/modellitellm.py:52: UserWarning: 
    Custom system prompt was provided which overrides the default system prompt. We assume that you know what you are doing.  
    . Default prompt for zerox is:
 
    Convert the following PDF page to markdown.
    Return only the markdown with no explanation text.
    Do not exclude any content from the page.
    
  warnings.warn(f"{Messages.CUSTOM_SYSTEM_PROMPT_WARNING}. Default prompt for zerox is:\n {DEFAULT_SYSTEM_PROMPT}")


Processing dataset/DATA2. e-F@ctory Model Line_Robot-Vision간 모델링 및 캘리브레이션 방법.pdf with zerox...
Processing dataset/DATA3. AMR 접속방법.pdf with zerox...
Processing dataset/DATA4. AMR 스캐너 에러 조치.pdf with zerox...
Processing dataset/DATA5. AMR 충전 실패 조치 방법.pdf with zerox...
Processing dataset/DATA6. 미쓰비시 e-Factory Model Line_메뉴얼_레이저_200319.pdf with zerox...
Processing dataset/DATA7. Trouble Shooting_200423.pdf with zerox...


In [ ]:
pymu_results_gathered = [
    pymupdf4llm.to_markdown(pdf_path, page_chunks=True, show_progress=False)
    for pdf_path in pdf_paths
]

In [8]:
def get_splitter() -> MarkdownHeaderTextSplitter:
    _target_headers = [
        ("#", "#"),
        ("##", "##"),
        ("###", "###"),
        ("####", "####"),
        ("#####", "#####"),
        ("######", "######"),
    ]
    return MarkdownHeaderTextSplitter(headers_to_split_on=_target_headers, strip_headers=False)


def split_pages_into_document_bundles(result: list[dict] | ZeroxOutput) -> list[dict]:
    # Result of pymupdf4llm
    if isinstance(result, list):
        spl = get_splitter()
        doc_bundles = []
        for page in result:
            docs = spl.split_text(page["text"])
            for doc in docs:
                doc_bundles.append(
                    {
                        "file_name": Path(page["metadata"]["file_path"]).name,
                        "page_number": page["metadata"]["page"],
                        "document": doc,
                    }
                )
        print(f"pymupdf4llm: {len(result)} pages split into {len(doc_bundles)} document bundles.")
    # Result of pyzerox (OCR)
    elif isinstance(result, ZeroxOutput):
        spl = get_splitter()
        doc_bundles = []
        for page in result.pages:
            docs = spl.split_text(page.content)
            for doc in docs:
                doc_bundles.append(
                    {
                        "file_name": Path(result.file_name).name,
                        "page_number": page.page,
                        "document": doc,
                    }
                )
        print(f"pyzerox: {len(result.pages)} pages split into {len(doc_bundles)} document bundles.")
    else:
        raise ValueError("The type of argument 'result' must be either a dict or ZeroxOutput.")

    return doc_bundles


def prepend_info_to_documents(
    document_bundles: list[dict],
    prepend_file_name: bool = True,
    prepend_metadata: bool = True,
) -> list[Document]:
    # Return only the document from document_bundles
    if not prepend_file_name and not prepend_metadata:
        return [doc["document"] for doc in document_bundles]

    doc_bundles_ = deepcopy(document_bundles)
    if prepend_metadata:
        for doc_bundle in doc_bundles_:
            doc = doc_bundle["document"]
            for header, header_content in reversed(doc.metadata.items()):
                if header_content not in doc.page_content:
                    doc.page_content = f"{header} {header_content}\n{doc.page_content}"
    if prepend_file_name:
        for doc_bundle in doc_bundles_:
            doc = doc_bundle["document"]
            doc.page_content = (
                f"File name: '{doc_bundle['file_name']}'\n"
                f"Page number: {doc_bundle['page_number']}\n"
                f"{doc.page_content}"
            )
    return [doc["document"] for doc in doc_bundles_]

In [9]:
zerox_all_docs = []
pymu_all_docs = []

for zerox_result, pymu_result in zip(zerox_results_gathered, pymu_results_gathered):
    zerox_docs = split_pages_into_document_bundles(zerox_result)
    pymu_docs = split_pages_into_document_bundles(pymu_result)
    zerox_docs_with_info = prepend_info_to_documents(zerox_docs)
    pymu_docs_with_info = prepend_info_to_documents(pymu_docs)
    zerox_all_docs += zerox_docs_with_info
    pymu_all_docs += pymu_docs_with_info

pyzerox: 54 pages split into 69 document bundles.
pymupdf4llm: 54 pages split into 91 document bundles.
pyzerox: 36 pages split into 46 document bundles.
pymupdf4llm: 36 pages split into 42 document bundles.
pyzerox: 2 pages split into 2 document bundles.
pymupdf4llm: 2 pages split into 4 document bundles.
pyzerox: 6 pages split into 13 document bundles.
pymupdf4llm: 6 pages split into 7 document bundles.
pyzerox: 6 pages split into 9 document bundles.
pymupdf4llm: 6 pages split into 6 document bundles.
pyzerox: 13 pages split into 17 document bundles.
pymupdf4llm: 13 pages split into 19 document bundles.
pyzerox: 1 pages split into 1 document bundles.
pymupdf4llm: 1 pages split into 1 document bundles.


In [10]:
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)

vector_store_txt = FAISS.from_documents(
    documents=pymu_all_docs,
    embedding=embeddings,
)
vector_store_ocr = FAISS.from_documents(
    documents=zerox_all_docs,
    embedding=embeddings,
)

In [11]:
RETRIEVE_K = 8

retriever_txt = vector_store_txt.as_retriever(search_kwargs={"k": RETRIEVE_K})
retriever_ocr = vector_store_ocr.as_retriever(search_kwargs={"k": RETRIEVE_K})

In [12]:
result = retriever_ocr.invoke(
    "로봇과 비전간의 캘리브레이션을 어떻게 하는지 알려줘 그리고 캘리브레이션이 무엇인지도 설명해줘"
)
for d in result:
    print(d.page_content)
    break

File name: 'DATA2. e-F@ctory Model Line_Robot-Vision간 모델링 및 캘리브레이션 방법.pdf'
Page number: 4
# 1. Robot – Vision 간 캘리브레이션  
캘리브레이션이란? 카메라 화면 상의 물체 위치정보 변화를량과 로봇의 실제 이동 거리간의 상관관계를 정의하고, 카메라 화면 상에서 1픽셀의 차이가 실제 로봇에서는 얼마나 움직여야 하는지 정의한다.  
![Camera and Robot](image)  
※ 카메라 화면 상에서 1픽셀 (1cm=37.8px, 1px=0.026cm)  
≠  
※ 로봇이 실제로 움직인 거리  
캘리브레이션: 카메라 화면에서 보이는 1픽셀과 실제 로봇이 움직인 거리를 맞추는 작업  
→ 로봇으로 P1에서 P2로 1.04mm를 움직였지만, Vision상에서는 0.78mm를 움직인걸로 차이가 발생한다.  
© Mitsubishi Electric Corporation  
Field Engineering Group


In [13]:
prompt_text = """
    You are an assistant for answering questions based on equipment manuals.
    Use the following retrieved context to answer the question.
    If the answer is unclear, try to deduce it from the provided contexts, or state that you don't know.
    Avoid using the context if it appears irrelevant or deteriorated.
    Provide your answer in Korean and format it as Markdown if applicable.
    If you are unsure about the context, you can ask for the file name to refer to.

    ## Context 1:
    {context_ocr}

    ## Context_2:
    {context_txt}

    ## Question:
    {question}

    ## Answer:
"""
prompt = PromptTemplate.from_template(strip_prompt(prompt_text))

In [14]:
llm = ChatOpenAI(model_name=MODEL_NAME, temperature=0.1)

In [15]:
chain = (
    {
        "context_ocr": retriever_ocr,
        "context_txt": retriever_txt,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
question = "가공기의 공구길이를 보정하는 방법을 알려줘, 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

가공기의 공구길이를 보정하는 방법은 다음과 같습니다:

1. 측정하고자 하는 공구를 선택합니다.
2. 수동모드(HANDLE)를 선택하고 Z축을 내려 기준 블록(또는 셋탑바)에 접촉시킵니다.
3. 아래의 방법으로 공구 길이를 계산한 후 그 값을 공구의 옵셋값 길이에 입력합니다.
   - A: 기계 원점으로부터 테이블 상면까지의 거리
   - B: 접촉 시 Z축 기계 좌표치
   - C: 공구 길이
   - D: 기준블럭 길이
   - 공구길이 (C) = A - B - D
4. 측정 완료 후 Z축은 원점복귀 합니다.

이 방법은 'DATA1. KT420(L) - 조작설명서 (MITSUBISH)_17 0420 - 완료.pdf' 문서의 38페이지를 참고하였습니다.


In [17]:
# 그림을 보고 답해야하는 질문
question = "가공기 내외부 구성 요소를 알려줘, 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

가공기 내외부 구성 요소는 다음과 같습니다:

- **외형도**: ELE BOX, TOWER LAMP, OIL PUMP, OPERATION PANEL, COOLANT TANK, FRONT DOOR
- **내부 구조 및 축 방향**: MAGAZINE, COLUMN, HEAD, TABLE, SADDLE, BED

이 정보는 'DATA1. KT420(L) - 조작설명서 (MITSUBISH)_17 0420 - 완료.pdf' 문서의 7페이지와 8페이지를 참고하였습니다.


In [18]:
question = "캘리브레이션이 무엇인지도 설명해줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

캘리브레이션이란 카메라 화면 상의 물체 위치정보 변화량과 로봇의 실제 이동 거리 간의 상관관계를 정의하고, 카메라 화면 상에서 1픽셀의 차이가 실제 로봇에서는 얼마나 움직여야 하는지를 정의하는 작업입니다. 이 과정은 카메라 화면에서 보이는 1픽셀과 실제 로봇이 움직인 거리를 맞추는 작업으로, 예를 들어 로봇이 P1에서 P2로 1.04mm를 움직였지만, Vision상에서는 0.78mm를 움직인 것으로 차이가 발생할 수 있습니다.

이 설명은 'DATA2. e-F@ctory Model Line_Robot-Vision간 모델링 및 캘리브레이션 방법.pdf' 문서의 4페이지를 참고하였습니다.


In [19]:
question = "로봇과 비전간의 캘리브레이션을 어떻게 하는지 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

로봇과 비전 간의 캘리브레이션은 다음과 같은 절차로 진행됩니다:

1. **수평 및 높이 조정**: 먼저, Vision과 대상물(컨베이어) 간의 높이 및 수평을 맞춥니다. 수평자는 기울기를 맞추는 데 사용되며, 높이는 350mm로 설정합니다. (출처: 페이지 5)

2. **캘리브레이션 정의**: 카메라 화면 상의 물체 위치 정보 변화량과 로봇의 실제 이동 거리 간의 상관관계를 정의합니다. 카메라 화면에서 1픽셀의 차이가 실제 로봇에서는 얼마나 움직여야 하는지를 정의합니다. (출처: 페이지 4)

3. **캘리브레이션 데이터 생성 및 적용**: 캘리브레이션은 1개의 모델에서 1회만 진행하여 데이터를 생성하고, 해당 데이터를 다른 모델에 복사/붙여넣기 하여 사용합니다. (출처: 페이지 19)

4. **로봇 이동량 기입**: World Coordinate에 로봇의 이동량을 기입합니다. 예를 들어, P2는 P1 기준으로 X가 50만큼, P3는 P1 기준으로 Y가 50만큼 이동합니다. (출처: 페이지 16)

이 절차는 'DATA2. e-F@ctory Model Line_Robot-Vision간 모델링 및 캘리브레이션 방법.pdf' 문서의 페이지 4, 5, 16, 19를 참고하였습니다.


In [20]:
question = "AMR 접속이 끊겼을때, IP주소 확인과 재할당 방법에 대해서 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

AMR 접속이 끊겼을 때 IP주소 확인과 재할당 방법은 다음과 같습니다:

### IP주소 확인 방법
1. AGV 독립 wifi에 접속합니다.
2. `mir.com`에 접속합니다.
3. ID는 `admin`을 사용하고, 비밀번호는 입력하지 않습니다.
4. 메뉴에서 `System` -> `System` -> `wifi 설정`으로 이동합니다.

### IP주소 재할당 방법
- IP 대역대가 `10.100.xxx.n`이 아니라면, Disconnect 후 Connect를 통해 `10.100.xxx.n`으로 자동 할당됩니다.

이 정보는 'DATA3. AMR 접속방법.pdf' 문서의 1페이지를 참고하였습니다.


In [21]:
question = "AMR 이동은 가능한데, 도킹 포지션에서 도킹일 안될 경우 해결 방법을 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

AMR 이동은 가능하지만 도킹 포지션에서 도킹이 안 되는 경우, 다음의 해결 방법을 참고하세요:

1. AMR 하단 전원버튼을 통해 재부팅을 진행합니다.
2. 에러가 반복될 경우, Laser Scanner 접촉 불량을 의심하고 Scanner USB를 재접속합니다.
3. AMR 미니PC를 확인하기 위해 후면부 커버를 분리합니다.
4. PC의 USB 케이블 접속 상태를 확인하고 재장착합니다.

이 정보는 'DATA4. AMR 스캐너 에러 조치.pdf' 문서의 3페이지를 참고했습니다.


In [22]:
question = "AMR이 충전기 도킹이 되었는데, 충전에 실패할 경우 어떻게 해결하는지 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

AMR이 충전기 도킹에 성공했지만 충전에 실패한 경우, 다음과 같은 조치 방법을 따르세요:

1. AMR을 충전기 20cm 내에 방향을 맞춰 배치합니다.
2. AMR 설정 웹페이지에 접속합니다.
3. AMR의 충전기를 클릭합니다.
4. Edit를 클릭하고, 5번 실행 전 값을 확인하기 위해 사진을 찍어둡니다.
5. Detect market를 클릭하여 AMR의 충전기 위치를 재조정합니다.

이 정보는 'DATA5. AMR 충전 실패 조치 방법.pdf' 문서의 2페이지와 3페이지를 참고했습니다.


In [23]:
question = "레이저 장비에 정면 도어열림 알람이 발생했을때, 어떻게 해결하는지 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

정면 도어열림 알람이 발생했을 때 해결 방법은 다음과 같습니다:

1. 에어가 공급되지 않아 도어가 열리지 않는 경우 에어 공급을 확인합니다.
2. 에어 스피드 컨트롤러가 너무 조여서 실린더를 동작시킬 수 없거나 현저히 느리게 동작하는지 확인합니다. (일정 시간이 경과할 경우 타임오버로 알람이 발생합니다.)
3. 에어 공급용 솔레노이드 밸브의 정상 동작을 확인합니다.
4. 도어 열림 확인용 실린더 센서 2포인트가 정상 동작하는지 확인합니다.

이 정보는 'DATA6. 미쓰비시 e-Factory Model Line_메뉴얼_레이저_200319.pdf' 문서의 11페이지를 참고하였습니다.


In [24]:
question = "레이저 장비에 정면 도어닫힘 알람이 발생했을때, 어떻게 해결하는지 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

정면 도어닫힘 알람이 발생했을 때 해결 방법은 다음과 같습니다:

1. 에어가 공급되지 않아 도어가 닫히지 않는 경우 에어 공급을 확인합니다.
2. 에어 스피드 컨트롤러가 너무 조여서 실린더를 동작시킬 수 없거나 현저히 느리게 동작하는지 확인합니다. (일정 시간이 경과할 경우 타임오버로 알람이 발생합니다.)
3. 에어 공급용 솔레노이드 밸브의 정상 동작을 확인합니다.
4. 도어 열림 확인용 실린더 센서 2포인트가 정상 동작하는지 확인합니다.

이 정보는 'DATA6. 미쓰비시 e-Factory Model Line_메뉴얼_레이저_200319.pdf' 문서의 11페이지를 참고하였습니다.


In [25]:
question = "레이저 장비에 컷팅 클램프 ON 알람이 발생했을때, 어떻게 해결하는지 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

컷팅 클램프 ON 알람이 발생했을 때의 해결 방법은 다음과 같습니다:

1. 워크가 공급되지 않거나 위치 이상으로 실린더의 마그네트 센서가 감지 불량이 아닌지 확인합니다.
2. 에어가 공급되지 않아 클램프가 동작되지 않는 경우 에어 공급을 확인합니다.
3. 에어 스피드 컨트롤러가 너무 조여서 실린더를 동작시킬 수 없거나 현저히 느리게 동작하는지 확인합니다. (일정 시간이 경과할 경우 타임오버로 알람이 발생합니다.)
4. 에어 공급용 솔레노이드 밸브의 정상 동작을 확인합니다.
5. 컷팅 클램프 확인용 실린더 센서 2포인트가 정상 동작하는지 확인합니다.

이 정보는 'DATA6. 미쓰비시 e-Factory Model Line_메뉴얼_레이저_200319.pdf' 문서의 11페이지와 12페이지를 참고하였습니다.


In [26]:
# 좀 막연한 질문이 들어온 경우
question = "기계 크기는?"
response = chain.invoke(question)
print(response)

기계 크기는 다음과 같습니다:

- KT420: 기계 높이 2655mm, 소요 바닥 면적 1760mm x 2520mm
- KT420L: 기계 높이 2655mm, 소요 바닥 면적 2064mm x 2520mm


In [37]:
# 세부 사항에 대한 질문이 들어온 경우
question = "가공기 소요 동력을 세부적으로 설명해줘"
response = chain.invoke(question)
print(response)

가공기의 소요 동력에 대한 세부 사항은 다음과 같습니다:

- **전원**: 220V±10%
- **정격 전원 용량**: 15.6 kVA
- **공압**: 5 kg/㎠ 이상

이 정보는 'DATA1. KT420(L) - 조작설명서 (MITSUBISH)_17 0420 - 완료.pdf' 파일의 5페이지에 기재되어 있습니다.


In [28]:
question = "레이저 장비에 마킹 클램프 ON 알람이 발생했을때, 어떻게 해결하는지 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

마킹 클램프 ON 알람이 발생했을 때 해결 방법은 다음과 같습니다:

1. 워크가 공급되지 않거나 위치 이상으로 실린더의 마그네트 센서가 감지 불량이 아닌지 확인합니다.
2. 에어가 공급되지 않아 클램프가 동작되지 않는 경우 에어 공급을 확인합니다.
3. 에어 스피드 컨트롤러가 너무 조여서 실린더를 동작시킬 수 없거나 현저히 느리게 동작하는지 확인합니다. (일정 시간이 경과할 경우 타임오버로 알람이 발생합니다.)
4. 에어 공급용 솔레노이드 밸브의 정상 동작을 확인합니다.
5. 컷팅 클램프 확인용 실린더 센서 2포인트가 정상 동작하는지 확인합니다.

이 정보는 'DATA6. 미쓰비시 e-Factory Model Line_메뉴얼_레이저_200319.pdf' 문서의 12페이지를 참고했습니다.


In [29]:
question = "레이저 장비에 MOS전진 알람이 발생했을때, 어떻게 해결하는지 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

레이저 장비에 MOS전진 알람이 발생했을 때 해결 방법은 다음과 같습니다:

1. 에어가 공급되지 않아 클램프가 동작되지 않는 경우 에어 공급을 확인합니다.
2. 에어 스피드 컨트롤러가 너무 조여서 실린더를 동작시킬 수 없거나 현저히 느리게 동작하는지 확인합니다. (일정 시간이 경과할 경우 타임오버로 알람이 발생합니다.)
3. 에어 공급용 솔레노이드 밸브의 정상 동작을 확인합니다.
4. 동작 확인용 실린더 센서가 정상 동작하는지 확인합니다.

이 정보는 'DATA6. 미쓰비시 e-Factory Model Line_메뉴얼_레이저_200319.pdf' 문서의 12페이지를 참고하였습니다.


In [30]:
question = "레이저 장비에 마킹 데이터 오류 알람이 발생했을때, 어떻게 해결하는지 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

마킹 데이터 오류 알람이 발생했을 때는 OPC UA로부터 반복해서 읽어온 마킹 데이터가 2회 이상 상이할 경우 발생합니다. 이 경우에는 제작사에 문의하는 것이 권장됩니다. 이 정보는 'DATA6. 미쓰비시 e-Factory Model Line_메뉴얼_레이저_200319.pdf' 문서의 13페이지를 참고하였습니다.


In [31]:
question = "레이저 장비에 측면 또는 후면 도어 열림 알람이 발생했을때, 어떻게 해결하는지 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

레이저 장비에 측면 또는 후면 도어 열림 알람이 발생했을 때 해결 방법은 다음과 같습니다:

- 원점복귀 또는 운전 중 측면 또는 후면 도어를 열 경우에 발생합니다.
- 도어 닫힘 감지용 센서가 이상 없는지 확인합니다.

이 내용은 'DATA6. 미쓰비시 e-Factory Model Line_메뉴얼_레이저_200319.pdf' 문서의 13페이지를 참고하였습니다.


In [32]:
question = "포장기에서 Vision 인식이 제대로 되지 않을 경우 어떻게 대처하면 되는지 알려줘. 무슨 문서의 몇 페이지를 참고했는지 알려줘."
response = chain.invoke(question)
print(response)

포장기에서 Vision 인식이 제대로 되지 않을 경우, 다음과 같이 대처할 수 있습니다:

1. GOT에서 Bypass 모드를 선택합니다.
2. 한 방향으로 15개씩 적재하여 재기동합니다. (봉투 투입구가 작업자 향하게)
3. 근본적인 조치는 비전 프로그램 검토가 필요하므로 HRC 대응 요청이 필요합니다.

이 정보는 'DATA7. Trouble Shooting_200423.pdf' 문서의 1페이지를 참고하였습니다.


In [38]:
question = "레이저 장비에서 컷팅 시 덜 잘려진 조각이 생성되면 어떻게 해야하나요?"
response = chain.invoke(question)
print(response)

레이저 장비에서 컷팅 시 덜 잘려진 조각이 생성될 경우, 다음과 같은 조치를 고려할 수 있습니다:

1. **레이저 출력 확인**: 컷팅 파라미터 설정에서 레이저 출력(Laser Power)을 확인하고, 필요에 따라 출력을 조정합니다. 레이저 출력이 충분하지 않으면 절단이 제대로 이루어지지 않을 수 있습니다.

2. **이동 속도 조정**: 컷팅 구간에서의 이동 속도(Speed)를 확인하고, 필요에 따라 속도를 조정합니다. 이동 속도가 너무 빠르면 절단이 덜 될 수 있습니다.

3. **컷팅 경로 확인**: 컷팅이 시작되는 X, Y 좌표(Start X, Start Y)와 잘라낼 아크릴 워크의 크기(Size X, Size Y)를 확인하여 설정이 정확한지 확인합니다.

4. **추가 이동 거리 설정**: 모서리의 확실한 절단을 위해 Extra 설정을 확인하고, 필요에 따라 조정합니다. 이 설정은 실제 아크릴 사이즈보다 약간 이동하는 거리입니다.

5. **장비 점검**: 장비의 상태를 점검하여 레이저의 초점이 맞는지, 장비에 이상이 없는지 확인합니다.

이러한 조치를 통해 덜 잘려진 조각 문제를 해결할 수 있습니다. 추가적인 문제가 지속될 경우, 장비 제조사에 문의하는 것이 좋습니다.


In [36]:
question = "레이저 장비에서 컷팅 시 덜 잘려진 조각이 생성되면 어떻게 해야하나요?  트러블슈팅 가이드를 참고해주세요."
response = chain.invoke(question)
print(response)

레이저 장비에서 컷팅 시 덜 잘려진 조각이 생성될 경우, 다음과 같은 조치를 취할 수 있습니다:

1. 컷팅쪽 경통을 풀어, 반사판의 그을음을 닦아줍니다.
2. 수동으로 레이저를 ON 시켜 초점이 한 가운데에 생성되는지 확인합니다.
3. 재조립을 진행합니다.

이러한 조치를 통해 문제를 해결할 수 있습니다.


In [34]:
question = "NC장비에서 안착 불량 에러시 대처?"
response = chain.invoke(question)
print(response)

NC 장비에서 안착 불량 에러가 발생했을 때의 대처 방법은 다음과 같습니다:

1. NC 가공기의 도어를 열고 Unclamp 하여 아크릴을 정위치에 놓습니다.
2. NC 가공기의 "Cycle Start" 버튼을 눌러서 재시작합니다.
3. NC GOT에서 자동 재기동 버튼을 눌러줍니다.

이 방법을 통해 안착 불량 문제를 해결할 수 있습니다.


In [45]:
# Save FAISS result
FAISS_SAVE_DIR = "faiss/"

vector_store_txt.save_local(FAISS_SAVE_DIR, index_name="demo_rag_txt")
vector_store_ocr.save_local(FAISS_SAVE_DIR, index_name="demo_rag_ocr")

In [46]:
# Load test
faiss_txt_loaded = FAISS.load_local(
    FAISS_SAVE_DIR,
    embeddings=OpenAIEmbeddings(model=EMBEDDING_MODEL),
    index_name="demo_rag_txt",
    allow_dangerous_deserialization=True,
)
faiss_ocr_loaded = FAISS.load_local(
    FAISS_SAVE_DIR,
    embeddings=OpenAIEmbeddings(model=EMBEDDING_MODEL),
    index_name="demo_rag_ocr",
    allow_dangerous_deserialization=True,
)
retriever_txt_loaded = faiss_txt_loaded.as_retriever(search_kwargs={"k": RETRIEVE_K})
retriever_ocr_loaded = faiss_ocr_loaded.as_retriever(search_kwargs={"k": RETRIEVE_K})